https://thecleverprogrammer.com/2021/03/03/spotify-recommendation-system-with-machine-learning/ used as a guide for what kind of pre-processing we should do. The rest comes from our knowledge from CPSC 330

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from joblib import dump

data = pd.read_csv("spotify.csv")

# Save the song names to a csv we can use in Flask to determine the names of recommended songs 
# based on the results from the Kmeans and NearestNeighbors models
# IMPORTANT: DO NOT CHANGE THE ORDER OF THE DATASET OTHERWISE THE INDICES USED TO RETRIEVE SONG IDs WILL RETURN THE WRONG ONES
song_ids = data['id']
song_ids.to_csv('song_ids.csv', index=False)

In [2]:
# data.head()

Select the relevant columns. We're choosing to do a more numerical analysis based on song features that can be measured in tempo, etc to determine which songs are similar. Year is also included since songs of a certain period tend to be more similar.

Also pre-process the numerical columns such that they are scaled between 0 to 1, so that no individual feature 'over-powers' the other features with a difference in magnitude of scale.

In [3]:
selected_columns = ['acousticness', 'danceability', 'energy', 
                    'instrumentalness', 'liveness', 'loudness', 
                    'speechiness', 'tempo', 'valence', 'popularity', 'year']
data = data[selected_columns]

# Scale the data and save it so the scaled dataset can be used in Flask
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=selected_columns) # Make df to fix Flask warning about training without labels
data_scaled.to_csv('data_scaled.csv', index=False)

A KMeans model clusters the training set into the number of clusters specified. It also assumes clusters are spherical. DBScan could be an alternative but tuning the parameters properly could be difficult, resulting in worse predictions. KMeans also scales better to larger datasets. 

In [4]:
# TODO: # Optimize the number of clusters using the elbow/silhouette method
k = 10 
kmeans = KMeans(n_clusters=k, n_init=10)
kmeans.fit(data_scaled)

KMeans(n_clusters=10, n_init=10)

In [5]:
num_recommended_songs = 5
nn = NearestNeighbors(n_neighbors=num_recommended_songs)
nn.fit(data_scaled)

NearestNeighbors()

https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
Saving to joblib for use in Flask without needing to re-train the entire model on start-up

In [6]:
dump(scaler, 'scaler.joblib')
dump(kmeans, 'kmeans.joblib')
dump(nn, 'nearest_neighbors.joblib')

['nearest_neighbors.joblib']